In [1]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pickle

import pandas as pd
import numpy as np
import re
import os
import eli5

/Users/lucy/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [7]:
def postprocessing(s):
    s[0] = re.split(r'[\([s]*[A-Z0-9\s]+[\s]*\)]|[\s]+', str(s[0]).upper())
    s[1] = re.sub(r'[\[|\]]','',s[1])
    s[2] = re.sub(r'[\[|\]]','',s[2])
    # parentheses, special symbols, ignore words
    labs = []
    for term in s[0]:
        found = False
        for k, tag in zip(str(s[1]).split(','),str(s[2]).split(',')):
            if re.sub(r'[^A-Z0-9\.]','',term) == re.sub(r'[^A-Z0-9\.]','',k):
                labs.append(term+'|'+tag)
                found = True
                break
        if not found:
            tag = ''
            labs.append(term+'|'+tag)
            
    # append tags to unlabeled terms.

    #VALUE: check special symbols
    i = 0
    while(i < len(labs) and (labs[i].split('|')[1]=='' or labs[i].split('|')[1]=='O')):
        if re.search(r'[\#\$\%\=\>\<]',labs[i].split('|')[0]):
            labs[i] = labs[i].split('|')[0] + '|' + 'VALUE'
        i = i + 1
        if  i < len(labs) and labs[i].split('|')[1]!='' and labs[i].split('|')[1]!='O':
            break  
    
    #LAB NAMES: check unlabeled terms
    i = 0
    while(i < len(labs) and (labs[i].split('|')[1]=='' or labs[i].split('|')[1]=='O')):
        labs[i] = labs[i].split('|')[0] + '|' + 'O'
        i = i + 1
        if  i < len(labs) and labs[i].split('|')[1]!='' and labs[i].split('|')[1]!='O':
            break

    #LAB NAMES: check unlabeled terms
    while( i < len(labs) and labs[i].split('|')[1]!='VALUE'):
        i = i+1
        
    while(i < len(labs) and (labs[i].split('|')[1]=='VALUE' or labs[i].split('|')[1]=='')):
        labs[i] = labs[i].split('|')[0] + '|' + 'VALUE'
        i = i + 1
        if  i < len(labs) and labs[i].split('|')[1]!='VALUE' and labs[i].split('|')[1]!='':
            break

    #REF: check parentheses
    for i in range(len(labs)):
        term = labs[i]
        if term.split('|')[1] == '' and re.search(r'[\(|\)]',term.split('|')[0]):
            labs[i] = term.split('|')[0] + '|REF' 
            
    #SCAN TAGS
    p_tag = 'O'
    for i in range(1,len(labs),1):
        term = labs[i]
        c_tag = term.split('|')[1]
        if c_tag!='' and p_tag != c_tag:
            p_tag = c_tag
        else:
            labs[i] = term.split('|')[0] +'|'+ p_tag
            
   
        
    return ','.join(labs)

In [6]:
%%time
df_test_data = pd.read_csv('data/train/train_round123_v31_structured_v14_label_test_tagged.csv')
df_test_data['parsed'] = df_test_data[['lab_result_1','lab_result_0','true_label']].apply(postprocessing,axis=1)
df_test_data.to_csv('data/test/train_round123_v31_structured_v14_postprocessing.csv',index=False)

CPU times: user 2.45 s, sys: 37.3 ms, total: 2.49 s
Wall time: 2.6 s
